In [1]:
# read configs
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))
from steps import conf

data_url = "https://liftingcast.com/meets/mvcjgjxtrlxi/results"

In [2]:
import polars as pl

In [3]:
reference_tables = "../reference-tables/usa-power-lifting-australia-tsf-team-champs-at-the-fort-2024.opl.csv"

df = pl.read_csv(reference_tables, skip_rows=5)

# Rename
renamed_columns = {column: conf.camel_to_snake(column) for column in df.columns}
team_champ_df = df.rename(mapping=renamed_columns)

# Filter for competitors
team_champ_competitors = team_champ_df.select(pl.col("name"))
team_champs_df = team_champ_df.select(["name", "birth_year", "weight_class_kg", "sex"])

In [4]:
raw_file_path = "../data/openpowerlifting-latest.parquet"
raw_df = pl.read_parquet(raw_file_path)

In [11]:
team_champ_competitors_history = raw_df.filter(pl.col("name").is_in(team_champ_competitors))
team_champ_competitors_history

name,sex,event,equipment,age,age_class,birth_year_class,division,bodyweight_kg,weight_class_kg,squat1_kg,squat2_kg,squat3_kg,squat4_kg,best3_squat_kg,bench1_kg,bench2_kg,bench3_kg,bench4_kg,best3_bench_kg,deadlift1_kg,deadlift2_kg,deadlift3_kg,deadlift4_kg,best3_deadlift_kg,total_kg,place,dots,wilks,glossbrenner,goodlift,tested,country,state,federation,parent_federation,date,meet_country,meet_state,meet_town,meet_name,event_year,age_rounded_to_half,min_birth_year,max_birth_year,year_of_birth,primary_key,origin_country
str,str,str,str,f64,str,str,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,str,str,str,str,str,date,str,str,str,str,i32,f64,f64,f64,i32,str,str
"""Tony Reinmuth""","""M""","""SBD""","""Raw""",null,null,null,"""Open""",149.3,"""120+""",315.0,335.0,345.0,null,345.0,195.0,202.5,-207.5,null,202.5,300.0,317.5,-332.5,null,317.5,865.0,2,465.75,478.92,452.98,91.97,"""Yes""","""Australia""",null,"""OceaniaPF""","""IPF""",2016-04-30,"""Australia""","""VIC""","""Melbourne""","""Pacific Invita…",2016,null,null,null,null,null,"""USA"""
"""Aaron Grant""","""M""","""SBD""","""Raw""",21.0,"""20-23""","""19-23""","""MR-O""",108.35,"""110""",190.0,205.0,-215.0,null,205.0,120.0,127.5,132.5,null,132.5,260.0,275.0,285.0,null,285.0,622.5,2,370.81,368.08,351.65,75.75,"""Yes""","""Australia""",null,"""USAPL""",null,2022-05-14,"""Australia""","""VIC""",null,"""Avia Performan…",2022,21.5,1998.5,2002.5,2000,"""aaron-grant-m-…","""Australia"""
"""Aaron Grant""","""M""","""SBD""","""Raw""",21.5,"""20-23""","""19-23""","""MR-Jr""",108.5,"""110""",207.5,220.0,-227.5,null,220.0,-127.5,127.5,135.0,null,135.0,280.0,302.5,-307.5,null,302.5,657.5,3,391.45,388.6,371.27,79.96,"""Yes""","""Australia""",null,"""USAPL""",null,2022-12-03,"""Australia""","""VIC""",null,"""Australia Nati…",2022,21.5,1998.5,2002.5,2000,"""aaron-grant-m-…","""Australia"""
"""Aaron Grant""","""M""","""SBD""","""Raw""",22.0,"""20-23""","""19-23""","""MR-O""",115.55,"""125""",210.0,225.0,232.5,null,232.5,130.0,137.5,142.5,null,142.5,270.0,290.0,305.0,null,305.0,680.0,1,395.6,394.62,377.8,80.37,"""Yes""","""Australia""",null,"""USAPL""",null,2023-04-22,"""Australia""",null,null,"""Bens Army Serv…",2023,22.5,1998.5,2002.5,2000,"""aaron-grant-m-…","""Australia"""
"""Aaron Grant""","""M""","""SBD""","""Raw""",22.5,"""20-23""","""19-23""","""MR-Jr""",118.9,"""125""",230.0,240.0,245.0,null,245.0,137.5,145.0,-150.0,null,145.0,295.0,315.5,-320.0,null,315.5,705.5,2,406.42,406.49,389.48,82.34,"""Yes""","""Australia""",null,"""USAPL""",null,2023-09-23,"""Australia""",null,null,"""Australian Jun…",2023,22.5,1998.5,2002.5,2000,"""aaron-grant-m-…","""Australia"""
"""Abbey Stronell…","""F""","""SBD""","""Raw""",19.5,"""20-23""","""19-23""","""FR-O""",65.85,"""67.5""",122.5,130.0,135.0,null,135.0,75.0,80.0,82.5,null,82.5,135.0,145.0,155.0,null,155.0,372.5,1,390.15,387.07,341.58,79.28,"""Yes""","""Australia""",null,"""USAPL""",null,2022-06-18,"""Australia""","""VIC""",null,"""The Strength F…",2022,19.5,2000.5,2004.5,2002,"""abbey-stronell…","""Australia"""
"""Abbey Stronell…","""F""","""SBD""","""Raw""",19.5,"""20-23""","""19-23""","""FR-O""",66.35,"""67.5""",127.5,137.5,-145.0,null,137.5,77.5,82.5,85.0,null,85.0,145.0,157.5,165.0,null,165.0,387.5,2,404.07,400.43,353.34,82.11,"""Yes""","""Australia""",null,"""USAPL""",null,2022-08-13,"""Australia""","""VIC""",null,"""Strength Cultu…",2022,19.5,2000.5,2004.5,2002,"""abbey-stronell…","""Australia"""
"""Abbey Stronell…","""F""","""SBD""","""Raw""",19.5,"""20-23""","""19-23""","""FR-O""",66.6,"""67.5""",130.0,137.5,-145.0,null,137.5,82.5,87.5,90.0,null,90.0,150.0,162.5,-177.5,null,162.5,390.0,4,405.78,401.92,354.63,82.45,"""Yes""","""Australia""",null,"""USAPL""",null,2022-12-03,"""Australia""","""VIC""",null,"""Australia Nati…",2022,19.5,2000.5,2004.5,2002,"""abbey-stronell…","""Australia"""
"""Abbey Stronell…","""F""","""SBD""","""Raw""",20.5,"""20-23""","""19-23""","""FR-O""",66.3,"""67.5""",137.5

In [21]:
team_champ_competitors_best_lifts_by_weight = team_champ_competitors_history.group_by(["name", "weight_class_kg"]).agg(pl.max("total_kg"))
team_champ_competitors_best_lifts_by_weight

name,weight_class_kg,total_kg
str,str,f64
"""Aengus William…","""105""",665.0
"""Chris Pham""","""83""",515.0
"""Constance Pand…","""67.5""",412.5
"""Edwina Vejayar…","""64""",337.0
"""Evangeline Mou…","""76""",435.0
"""Hung Phan""","""110""",830.0
"""India Stewart""","""57""",300.0
"""Jessica Stikov…","""100+""",407.5
"""Matthew Crunda…","""93""",698.5


In [22]:
team_champ_competitors_best_lifts_by_dots = team_champ_competitors_history.group_by(["name", "weight_class_kg"]).agg(pl.max("dots"))
team_champ_competitors_best_lifts_by_dots

name,weight_class_kg,dots
str,str,f64
"""Aaron Grant""","""110""",391.45
"""Brandon Refalo…","""67.5""",370.11
"""Claudia Coppol…","""58""",380.88
"""Constance Pand…","""64""",414.63
"""Emry Williams""","""120+""",379.78
"""Evangeline Mou…","""63""",389.87
"""Evangeline Mou…","""76""",422.64
"""India Stewart""","""57""",347.12
"""Jacob Schepis""","""83""",452.07


In [23]:
team_champs_df.join(
    team_champ_competitors_best_lifts_by_weight,
    on=["name", "weight_class_kg"],
    how="left",
).with_columns(
    rn=pl.col("total_kg").rank("ordinal", descending=True).over(["weight_class_kg", "sex"])
).sort(by=pl.col("rn")).partition_by(["weight_class_kg", "sex"])

[shape: (4, 6)
 ┌─────────────────┬────────────┬─────────────────┬─────┬──────────┬──────┐
 │ name            ┆ birth_year ┆ weight_class_kg ┆ sex ┆ total_kg ┆ rn   │
 │ ---             ┆ ---        ┆ ---             ┆ --- ┆ ---      ┆ ---  │
 │ str             ┆ str        ┆ str             ┆ str ┆ f64      ┆ u32  │
 ╞═════════════════╪════════════╪═════════════════╪═════╪══════════╪══════╡
 │ Carol Chew      ┆ 1978       ┆ 56              ┆ F   ┆ null     ┆ null │
 │ Thuy Vy Pham    ┆ 2000       ┆ 56              ┆ F   ┆ null     ┆ null │
 │ Claudia Coppola ┆ 1995       ┆ 56              ┆ F   ┆ null     ┆ null │
 │ Lina Tran       ┆ 1999       ┆ 56              ┆ F   ┆ 356.5    ┆ 1    │
 └─────────────────┴────────────┴─────────────────┴─────┴──────────┴──────┘,
 shape: (10, 6)
 ┌─────────────────────┬────────────┬─────────────────┬─────┬──────────┬──────┐
 │ name                ┆ birth_year ┆ weight_class_kg ┆ sex ┆ total_kg ┆ rn   │
 │ ---                 ┆ ---        ┆ ---       

In [24]:
team_champs_df.join(
    team_champ_competitors_best_lifts_by_dots,
    on=["name", "weight_class_kg"],
    how="left",
).with_columns(
    rn=pl.col("dots").rank("ordinal", descending=True).over(["weight_class_kg", "sex"])
).sort(by=pl.col("rn")).partition_by(["weight_class_kg", "sex"])

[shape: (4, 6)
 ┌─────────────────┬────────────┬─────────────────┬─────┬───────┬──────┐
 │ name            ┆ birth_year ┆ weight_class_kg ┆ sex ┆ dots  ┆ rn   │
 │ ---             ┆ ---        ┆ ---             ┆ --- ┆ ---   ┆ ---  │
 │ str             ┆ str        ┆ str             ┆ str ┆ f64   ┆ u32  │
 ╞═════════════════╪════════════╪═════════════════╪═════╪═══════╪══════╡
 │ Carol Chew      ┆ 1978       ┆ 56              ┆ F   ┆ null  ┆ null │
 │ Thuy Vy Pham    ┆ 2000       ┆ 56              ┆ F   ┆ null  ┆ null │
 │ Claudia Coppola ┆ 1995       ┆ 56              ┆ F   ┆ null  ┆ null │
 │ Lina Tran       ┆ 1999       ┆ 56              ┆ F   ┆ 415.4 ┆ 1    │
 └─────────────────┴────────────┴─────────────────┴─────┴───────┴──────┘,
 shape: (10, 6)
 ┌─────────────────────┬────────────┬─────────────────┬─────┬────────┬──────┐
 │ name                ┆ birth_year ┆ weight_class_kg ┆ sex ┆ dots   ┆ rn   │
 │ ---                 ┆ ---        ┆ ---             ┆ --- ┆ ---    ┆ ---  │
 │ s